In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc (
  region STRING,
  store_id STRING,
  store_name STRING,
  product_id STRING,
  product_name STRING,
  business_unit STRING,
  province STRING,
  mdq DOUBLE,
  proc_date DATE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc
WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc_v2)
""")

In [0]:
spark.sql(f""" 
INSERT INTO {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc (
    region,
    store_id,
    store_name,
    product_id,
    product_name,
    business_unit,
    province,
    mdq,
    proc_date
)
SELECT
    region,
    store_id,
    store_name,
    product_id,
    product_name,
    business_unit,
    province,
    mdq,
    proc_date
FROM  {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc_v2 WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc_v2)
"""
)